In [ ]:
#general definitions and imports
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import sys
sys.path.append('../../lib/python')

import os
import PySpice.Logging.Logging as Logging
logger = Logging.setup_logging()

import numpy as np
import matplotlib.pyplot as plt

import unittest

import SchemDraw as schemdraw
import SchemDraw.elements as elm

from pathlib import Path

from PySpice.Unit import *
from PySpice.Spice.Parser import SpiceParser
from PySpice.Spice.Netlist import SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Probe.Plot import plot
from PySpice.Math import *

## setup spice library path
directory_path = Path(os.path.abspath('')).resolve().parent.parent
spice_libraries_path = directory_path.joinpath("lib", "spice")
spice_library = SpiceLibrary(spice_libraries_path)

## setup path to kicad schema
directory_path = Path(os.path.abspath('')).resolve()
kicad_netlist_path = directory_path.joinpath( 'main', 'main.cir' )
NETLIST = str( kicad_netlist_path )

In [ ]:
## schema definitions
d = schemdraw.Drawing()
op = d.add(elm.Opamp)
d.add(elm.Line(xy=op.out, l=.75, rgtlabel='$v_{out}$'))

d.add(elm.Line('left', xy=op.in1, l=.75))
d.add(elm.Line('up', l=1.5))
d.add(elm.Dot)
R1 = d.add(elm.Resistor('left', label='$R_1$'))
R5 = d.add(elm.Resistor('down', label='$R_5$', rotation=90, rotation_mode='default'))
d.add(elm.Dot)
d.add(elm.Resistor('down', label='$R_6$', rotation=90, rotation_mode='default'))
d.add(elm.Ground)

l = d.add(elm.Line('left', xy=R1.end, l=1.5))
d.add(elm.Line('left', l=.5, lftlabel='$v_{in}$'))
d.add(elm.Line('down', xy=l.end, l=1.5))
POT = d.add(elm.Potentiometer('down'))
POT.add_label('$RV_{(R3/R4)}$', loc='center', ofst=[0,-1], align=('right','bottom'))
d.add(elm.Ground,xy=POT.end)
d.add(elm.Line('right', xy=POT.tap, tox=R5.end))
d.add(elm.Line('right', xy=POT.tap, tox=op.in2-.75))
d.add(elm.Line('up', l=0.5))
d.add(elm.Line('right', l=.75))
Rf = d.add(elm.Resistor('right', xy=R1.start, tox=op.out+.5, label='$R_2$'))
d.add(elm.Line('down', toy=op.out))
end = d.add(elm.Dot)



In [ ]:
diffamp = schemdraw.Drawing()
diffamp_op = diffamp.add(elm.Opamp)
diffamp.add(elm.Line(xy=op.out, l=.75))
diffamp.add(elm.Line('left', xy=op.in1, l=.75))
diffamp.add(elm.Line('up', l=1.5))
diffamp.add(elm.Dot)
diffamp_R1 = diffamp.add(elm.Resistor('left', label='$R_1$', lftlabel='$v_{1}$'))
diffamp_Rf = diffamp.add(elm.Resistor('right', xy=diffamp_R1.start, tox=diffamp_op.out+.5, label='$R_2$'))
diffamp.add(elm.Line('down', toy=diffamp_op.out))
diffamp.add(elm.Dot)

diffamp.add(elm.Line('left', xy=diffamp_op.in2, l=.75))
diffamp.add(elm.Line('down', l=1.5))
diffamp_dot = diffamp.add(elm.Dot)

diffamp_R3 = diffamp.add(elm.Resistor('right', label='$R_3$'))
diffamp.add(elm.Ground)
X = diffamp.add(elm.Resistor('left', xy=diffamp_dot.start, label='$R_4$', lftlabel='$v_{2}$'))


In [ ]:
class XU1(SubCircuitFactory):
    __name__ = 'XU1'
    __nodes__ = ( 'OUT', 'INVERTING', 'NON_INVERTING', 'POWER_NEGATIVE', 'POWER_POSITIVE' )
    def __init__(self):
        super().__init__()
        self.X('1', 'OPA2134', 'NON_INVERTING', 'INVERTING', 'POWER_POSITIVE', 'POWER_NEGATIVE', 'OUT' )

class VoltageDivider1(SubCircuitFactory):
    __name__ = 'voltage_divider_1'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, w=0.4):
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )

class VoltageDivider2(SubCircuitFactory):
    __name__ = 'voltage_divider_2'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, w=0.4):
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )

class VoltageDivider3(SubCircuitFactory):
    __name__ = 'voltage_divider_3'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, w=0.4):
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )

class VoltageDivider4(SubCircuitFactory):
    __name__ = 'voltage_divider_4'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=100@u_kΩ, w=0.4):
        super().__init__()
        self.R(1, 'n1', 'n2', R * w)
        self.R(2, 'n2', 'n3', R * (1.0-w) )

class JackOut(SubCircuitFactory):
    __name__ = 'jack_out'
    __nodes__ = ('n1', 'n2', 'n3' )
    def __init__(self, R=1@u_Ω):
        super().__init__()
        self.R(1, 'n1', 'n3', R)

# show the netlist
parser = SpiceParser(path=NETLIST)


In [ ]:
from sympy import symbols, sqrt, exp, diff, integrate, pprint, simplify, Eq, preview

from sympy import init_printing
#init_printing(use_latex='mathml')

R1, R2, R3, R4, Vin, Vin2, Vout = symbols('R1 R2 R3 R4 Vin Vin2, Vout', real=True)
eq_diff = (1+R2/R1) * (R4/(R4+R3)) * Vin2 - (R2/R1) * Vin 
eq_paralell_resistor =  1 /( (1/R1) + (1/R2) ) 


![module](https://img.shields.io/badge/module-mischer-yellow)
![status](https://img.shields.io/badge/status-work%20in%20progress-orange)

# *kontrast*

<a href="https://photos.app.goo.gl/pg6iZUC32rTKX5LC7"><img src="https://spielhuus.github.io/elektrophon/images/kontrast-logo-tmb.jpg" height="300px" align="right"></a>

***table of contents***

* [*about*](#about)
* [*construction*](#construction)
* [*calibration*](#calibration)
* [*usage*](#usage)
* [*references*](#lreferences)
* [*changelog*](#changelog)

<br/><br/>

## *about*

kontrast ist a simple mixer utility module. it can be used as a **mixer**, **attenuator** or **attenuverter** for ac or dc signals. the function depends on the patching and settings of the potentiometers. an attenuverter is used in different modules like the  Serge VCS, Maths and also in the polivoks filter. while this module can be used as a simple mixer you can also adjust cv signals. such a signal can be simply attenuated or inverted. when feeding in a positive singal slope (for example from an adsr). the signal will be turned to a neagative dc singal. if the inpout signal is ac coupled the signal will be inverted. the function is the setting on the potentiometer of the channel. when the potentiometer is at center position. the output signal is attenuated to zero. no output singal is present. when turning to the right. the signal will be stronger until the output signal is equal to the input signal. when turning the potentiomenter to the left, the output signal will be phase shifted by 180°. when you turn the potentiometer to full left the output signal will be the inverted input signal. all the inputs are mixed to the output signal; unless you connect the output of a channel. then this output does not accure on the mixed signal.

the attenuverter is build around a differential amplifier with an opamp. 

In [ ]:
diffamp.draw()

In [ ]:
Eq( Vout, eq_diff)


the circuit of the attenuverter is based on the opamp dfferential amplifer. the two resistors at the non-inverting inputs are replaced with a potentiometer. the potentiometer is configured as a voltage divider and replace R3 and R4 from the differential amplifier. when the position of the potentiometer is adjusted, the output voltage will change or invert. at the center position  the output is zero volrs. the response to the potentiometer is linear. we can make it an centered s-curve by adding two parallel resistors (R5,R6) to the potentiometer [[2][2]].

In [ ]:
d.draw()

a simulation with the resistance at RV 1 changed from 0-100%. first run is the linear result. the second run is done with the resistors R5 and R6 added. This results in a s curve. the shape of the curve can be changed with the values of the resistors.

In [ ]:
Vlin = []
Vlog = []

R1val = 100@u_kOhm
R2val = 100@u_kOhm
RVval = 100@u_kOhm
R5val = 33@u_kOhm
RVstep = 1@u_kOhm

for Rtemp in np.arange(1@u_Ohm, RVval, RVstep):
    R3t = Rtemp
    R4t = RVval - Rtemp
    Vtemp = eq_diff.subs(R1, R1val).subs(R2, R2val).subs(R3, R3t).subs(R4, R4t).subs(Vin, 5).subs(Vin2, 5)
    Vlin.append(Vtemp)

for Rtemp in np.arange(1@u_Ohm, RVval, RVstep):
    R3t =  eq_paralell_resistor.subs(R1, Rtemp).subs(R2, R5val)
    R4t = eq_paralell_resistor.subs(R1, RVval - Rtemp).subs(R2, R5val) 
    Vtemp = eq_diff.subs(R1, R1val).subs(R2, R2val).subs(R3, R3t).subs(R4, R4t).subs(Vin, 5) .subs(Vin2, 5)   
    Vlog.append(Vtemp)

figure, ax = plt.subplots(figsize=(8, 5))
ax.plot(Vlin)  # the pot position from 0 .. 1
ax.plot(Vlog)  # the pot position from 0 .. 1
ax.legend(('Vlin [V]', 'Vlog [V]'), loc=(.7, .8))
ax.grid()
ax.set_xlabel('gain [%]')
ax.set_ylabel('Vout [V]')
plt.tight_layout()
plt.show()

## construction

because the channel is silent at potentometer center position, it would be nice to use a potentometer that has a center decent. such potientometers are not easy to find. some examples are [[10][10]]


<img src="main/main.svg"/>


Kicad Schema!


In [ ]:
circuit = parser.build_circuit(ground='GND')
circuit.include(spice_library['LED2'])
circuit.include(spice_library['D1N4148'])
circuit.include(spice_library['BC547B'])
circuit.include(spice_library['OPA2134'])
circuit.include(spice_library['OPA2134d'])
circuit.V('1', '+15V', circuit.gnd, 15@u_V)
circuit.V('2', '-15V', circuit.gnd, -15@u_V)

for subcircuit in ( XU1(), VoltageDivider1(R=RVval,w=0), VoltageDivider2(R=RVval,w=0.5), VoltageDivider3(R=RVval,w=0.5), VoltageDivider4(R=RVval,w=0.5) ,JackOut()):
    circuit.subcircuit(subcircuit)

print(str(circuit))


simulation to check the attenuation function. 

In [ ]:
steps = np.arange( 1, 0, -0.01 )
results = []

for s in steps:

    circuit = parser.build_circuit(ground='GND')
    circuit.include(spice_library['LED2'])
    circuit.include(spice_library['D1N4148'])
    circuit.include(spice_library['BC547B'])
    circuit.include(spice_library['OPA2134'])
    circuit.include(spice_library['OPA2134d'])
    circuit.V('1', '+15V', circuit.gnd, 15@u_V)
    circuit.V('2', '-15V', circuit.gnd, -15@u_V)

    for subcircuit in ( XU1(), VoltageDivider1(R=RVval,w=s), VoltageDivider2(R=RVval,w=0.5), 
                                            VoltageDivider3(R=RVval,w=0.5), VoltageDivider4(R=RVval,w=0.5), JackOut()):
        circuit.subcircuit(subcircuit)

    simulator = circuit.simulator(temperature=25, nominal_temperature=25)
    analysis = simulator.operating_point()

    results.append( float( analysis['/MIX_1'] ) )

figure, ax = plt.subplots(figsize=(8, 5))
ax.plot(results)  # input
ax.legend(('Vout [V]'), loc=(.1, .8))
ax.grid()
ax.set_xlabel('wiper [%]')
ax.set_ylabel('[V]')

plt.tight_layout()
plt.show()


In [ ]:
parser = SpiceParser(path=NETLIST)
circuit = parser.build_circuit(ground='GND')
circuit.include(spice_library['LED2'])
circuit.include(spice_library['D1N4148'])
circuit.include(spice_library['BC547B'])
circuit.include(spice_library['OPA2134'])
circuit.include(spice_library['OPA2134d'])
circuit.V('1', '+15V', circuit.gnd, 15@u_V)
circuit.V('2', '-15V', circuit.gnd, -15@u_V)

for subcircuit in ( XU1(), VoltageDivider1(R=RVval,w=0.2), VoltageDivider2(R=RVval,w=0.2), 
                                        VoltageDivider3(R=RVval,w=0.2), VoltageDivider4(R=RVval,w=0.2), JackOut()):
    circuit.subcircuit(subcircuit)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.transient(step_time=10@u_us, start_time=10@u_ms, end_time=11@u_ms)

figure, ax = plt.subplots(figsize=(8, 5))

#ax.plot(analysis['Net-_J2-PadT_'])  # channel 1 input (signal)
#ax.plot(analysis['/MIX_1'])  # channel 1 output (signal)
#ax.plot(analysis['/MIX_2'])  # channel 2 output (silent)
ax.plot(analysis['Net-_J10-PadT_'])  # mixed output
ax.legend(('Vin [V]', 'V#1 [V]', 'V#2 [V]', 'Vmix [V]'), loc=(.1, .8))
ax.grid()
ax.set_xlabel('t [s]')
ax.set_ylabel('[V]')

plt.tight_layout()
plt.show()

test the led voltage

In [ ]:
parser = SpiceParser(path=NETLIST)
circuit = parser.build_circuit(ground='GND')
circuit.include(spice_library['LED2'])
circuit.include(spice_library['D1N4148'])
circuit.include(spice_library['BC547B'])
circuit.include(spice_library['OPA2134'])
circuit.include(spice_library['OPA2134d'])
circuit.V('1', '+15V', circuit.gnd, 15@u_V)
circuit.V('2', '-15V', circuit.gnd, -15@u_V)

for subcircuit in ( XU1(), VoltageDivider1(R=RVval,w=0.01), VoltageDivider2(R=RVval,w=0.1), 
                                        VoltageDivider3(R=RVval,w=0.2), VoltageDivider4(R=RVval,w=0.2), JackOut()):
    circuit.subcircuit(subcircuit)

simulator = circuit.simulator(temperature=25, nominal_temperature=25)
analysis = simulator.transient(step_time=1@u_us, start_time=10@u_ms, end_time=100@u_ms)

figure, ax = plt.subplots(figsize=(8, 5))

#ax.plot(analysis['/MIX_1'])  # channel 1 output (signal)
ax.plot(analysis['Net-_C9-Pad2_'])  # transistor input
ax.plot(analysis['Net-_D2-Pad2_'])  # transistor output
ax.legend(('Vin [V]', 'V#1 [V]', 'V#2 [V]', 'Vmix [V]'), loc=(.1, .8))
ax.grid()
ax.set_xlabel('t [s]')
ax.set_ylabel('[V]')

plt.tight_layout()
plt.show()

tests

In [ ]:
import unittest
class TestKontrast(unittest.TestCase):
    
    def test_add(self):
        circuit = parser.build_circuit(ground='GND')
        circuit.include(spice_library['LED2'])
        circuit.include(spice_library['D1N4148'])
        circuit.include(spice_library['BC547B'])
        circuit.include(spice_library['OPA2134'])
        circuit.include(spice_library['OPA2134d'])
        circuit.V('1', '+15V', circuit.gnd, 15@u_V)
        circuit.V('2', '-15V', circuit.gnd, -15@u_V)

        for subcircuit in ( XU1(), VoltageDivider1(R=RVval,w=0), VoltageDivider2(R=RVval,w=1.0), 
                                                VoltageDivider3(R=RVval,w=0.5), VoltageDivider4(R=RVval,w=0.5), JackOut()):
            circuit.subcircuit(subcircuit)
        
        simulator = circuit.simulator(temperature=25, nominal_temperature=25)
        analysis = simulator.operating_point()

        print( "\n\nMIX1: %f, MIX2: %f, MIX3: %f, MIX4: %f, OUT: %f" % 
            (float( analysis['/MIX_1'] ), float( analysis['/MIX_2'] ), 
            float( analysis['/MIX_3'] ), float( analysis['/MIX_4'] ), 
            float( analysis['Net-_J10-PadT_'] ) ) )

        self.assertEqual(float( analysis['Net-_J1-PadT_'] ), 5 )
        self.assertEqual(float( analysis['Net-_J2-PadT_'] ), 5 )
        self.assertEqual(float( analysis['Net-_J3-PadT_'] ), 5 )
        self.assertEqual(float( analysis['Net-_J4-PadT_'] ), 5 )
        self.assertAlmostEqual(float( analysis['/MIX_1'] ), -5, places=1 )
        self.assertAlmostEqual(float( analysis['/MIX_2'] ), 5, places=1)
        self.assertAlmostEqual(float( analysis['/MIX_3'] ), 0, places=1 )
        self.assertAlmostEqual(float( analysis['/MIX_4'] ), 0, places=1 )
        self.assertAlmostEqual(float( analysis['Net-_J10-PadT_'] ), 0, places=2 )

unittest.main(argv=[''], verbosity=2, exit=False)


## *calibration*

there is no calibration needed. but the potentiomenter knobs have to be aliged to center position.

## *usage*

the input jacks are wired to 5 volts when nothing is connected.

all channels are mixed to the out jack. when something is connected to the channel out, this channel is removed from the overall mix.

_mixer_
* connect the different channels from audio or cv sources to the in jacks.
* connect the out jack to something
* turn the pots clockwise to adjust the volume.

_attenuverter_
* connect all or a single channel. 
* when you turn the pot knob counter clockwise the signal is inverted.


## *references*

- [Op Amp Differential Amplifier][1] eCircuit  Center
- [Beginners' Guide to Potentiometers][2]  Rod Elliott (ESP)

[1]: http://www.ecircuitcenter.com/Circuits/opdif/opdif.htm
[2]: https://sound-au.com/pots.htm
[10]: https://www.mouser.ch/ProductDetail/ALPS/RK09L114001T?qs=Xb8IjHhkxj78e5CYccfZWg%3D%3D


## *changelog*



---
[![CC BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)](https://creativecommons.org/licenses/by-sa/4.0/)